# 1. Install Dependencies

pip install torch from https://pytorch.org/ based on your specs

In [1]:
#install YOLOv5
!git clone https://github.com/ultralytics/yolov5
    
#notes:
'''Cloning a GitHub repository means creating a local copy of a remote repository hosted on GitHub. 
git: This is the command-line tool for version control, commonly used for tracking changes in source
code and collaborating with others on software projects.

clone: This is a subcommand of Git that is used to create a copy of a remote Git repository on your local 
machine. When you run git clone, you are essentially downloading all the files and history from the remote repository.

https://github.com/ultralytics/yolov5: This is the URL of the Git repository you want to clone. In this case, you are 
trying to clone a repository named "yolov5" from the GitHub account "ultralytics
'''



fatal: destination path 'yolov5' already exists and is not an empty directory.


'Cloning a GitHub repository means creating a local copy of a remote repository hosted on GitHub. \ngit: This is the command-line tool for version control, commonly used for tracking changes in source\ncode and collaborating with others on software projects.\n\nclone: This is a subcommand of Git that is used to create a copy of a remote Git repository on your local \nmachine. When you run git clone, you are essentially downloading all the files and history from the remote repository.\n\nhttps://github.com/ultralytics/yolov5: This is the URL of the Git repository you want to clone. In this case, you are \ntrying to clone a repository named "yolov5" from the GitHub account "ultralytics\n'

In [2]:
!cd yolov5 & pip install -r requirements.txt

'''cd yolov5: This part of the command uses the cd command to change the current directory to a directory called "yolov5."
In many command-line interfaces, the cd command is used to navigate to a different directory or folder.

pip install -r requirements.txt: This part of the command uses pip, which is a package manager for Python, to install Python
packages listed in a file called "requirements.txt." The -r flag tells pip to read the package names
and versions from the specified file and install them.

Basically just checks if all the requirements are met in the system in whcih the code is running'''


'cd yolov5: This part of the command uses the cd command to change the current directory to a directory called "yolov5."\nIn many command-line interfaces, the cd command is used to navigate to a different directory or folder.\n\npip install -r requirements.txt: This part of the command uses pip, which is a package manager for Python, to install Python\npackages listed in a file called "requirements.txt." The -r flag tells pip to read the package names\nand versions from the specified file and install them.\n\nBasically just checks if all the requirements are met in the system in whcih the code is running'

# 2. Realtime Gauze Detection w/ 2 Webcams

In [6]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2  #Open-cv
import time

# YOLOv5

model = torch.hub.load('ultralytics/yolov5', 'custom', path='../yolov5-master/202318JUL.pt', force_reload=True)
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5x6', force_reload=True)

model.conf = 0.50
#model.max_det = 20
model.classes = [0]
model2.conf = 0.25
#model2.max_det = 20
model2.classes = [0]

onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, 0, 0, 0
condition = 7
cap1 = cv2.VideoCapture(0)
#Check this
cap2 = cv2.VideoCapture(1)

while True:
    ret, frame1 = cap1.read() #Read image frame-by-frame
    ret, frame2 = cap2.read()

    # Make detections
    pic = np.hstack((frame1, frame2))
    results = model(np.hstack((frame1, frame2))) #Parallel computation, no need to stack it using numpy
    results2 = model2(np.hstack((frame1, frame2)))

    a, b, c, d = 0, 0, 0, 0
    for detection in results.xyxy[0]:

        # Gauze detection
        if detection[5].item() in model.classes:
            if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                a += 1
            elif detection[2].item() > frame1.shape[1]:
                b += 1

    for detection in results2.xyxy[0]:
        # Human detection
        if detection[5].item() == 0:
            if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                c += 1
            elif detection[2].item() > frame1.shape[1]:
                d += 1

    image = np.squeeze(results.render())

    # Adding black border at the bottom
    borderedFrame = cv2.copyMakeBorder(image, 0, 350, 640 - frame1.shape[1], 640 - frame2.shape[1], cv2.BORDER_CONSTANT,
                                       value=[0, 0, 0])

    if c == 0:
        tempcount1 = a
        if tempcount1 != onscreenIn:
            frameCountIn += 1
            if frameCountIn == condition:
                if tempcount1 > onscreenIn:
                    countIn += tempcount1 - onscreenIn
                onscreenIn = tempcount1
        else:
            frameCountIn = 0
        cv2.putText(borderedFrame, 'On Screen =' + str(onscreenIn), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + str(countIn), (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                    2, cv2.LINE_AA)
    else:
        cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + 'Paused', (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Human Detected', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

    if d == 0:
        tempcount2 = b
        if tempcount2 != onscreenOut:
            frameCountOut += 1
            if frameCountOut == condition:
                if tempcount2 > onscreenOut:
                    countOut += tempcount2 - onscreenOut
                onscreenOut = tempcount2
        else:
            frameCountOut = 0
        cv2.putText(borderedFrame, 'On Screen =' + str(onscreenOut), (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total Out =' + str(countOut), (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
    else:
        cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total Out =' + 'Paused', (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Human Detected', (840, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # FPS Counter
    endTime = time.time()
    fps = 1 / (endTime - startTime)
    startTime = time.time()
    cv2.putText(borderedFrame, "FPS =" + str(round(fps, 1)), (540, 520), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                2, cv2.LINE_AA)

    # Gauze In-Play Counter
    countPlay = countIn - onscreenIn - countOut
    cv2.putText(borderedFrame, 'In Play =' + str(countPlay), (360, 650), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4,
                cv2.LINE_AA)

    cv2.imshow('Gauze', borderedFrame)

    if cv2.waitKey(10) & 0xFF == ord('q'): #q is to stop program
        break
    key = cv2.waitKey(10)
    user_input = input("Choose 'A' to chage onscreenIn,B for countIn, C for onscreenOut, D for countOut and E for countPlay")
    if user_input=='A':
        
    if key & 0xFF == ord('0'):
        onscreenIn += 1
    elif key & 0xFF == ord('1'):
        onscreenIn -= 1
    elif key & 0xFF == ord('2'):
        countIn += 1 
    elif key & 0xFF == ord('3'):
        countIn -= 1
    elif key & 0xFF == ord('4'):
        onscreenOut += 1
    elif key & 0xFF == ord('5'):
        onscreenOut -= 1
    elif key & 0xFF == ord('6'):
        countOut += 1
    elif key & 0xFF == ord('7'):
        countOut -= 1
    elif key & 0xFF == ord('8'):
        countPlay += 1
    elif key & 0xFF == ord('9'):
        countPlay -= 1
    elif key & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
YOLOv5  2023-9-15 Python-3.7.16 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
YOLOv5  2023-9-15 Python-3.7.16 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients, 209.6 GFLOPs
Adding AutoShape... 


In [5]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time
# YOLOv5
frame_count = 0
start_time = time.time()
avg_fps = 0


#model = torch.hub.load('ultralytics/yolov5', 'custom', path='../yolov5-master/202318JUL.pt', force_reload=True)
model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:/Users/Tee Chang Zen/Documents/OneDrive/UREKA Project/IGuaze/yolov5-master/202318JUL.pt', force_reload=True)
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5x6', force_reload=True)

model.conf = 0.50
#model.max_det = 20
model.classes = [0]
model2.conf = 0.25
#model2.max_det = 20
model2.classes = [0]

onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, 0, 0, 0
condition = 7
cap1 = cv2.VideoCapture(0)
#Check this
cap2 = cv2.VideoCapture(1)

while True:
    ret, frame1 = cap1.read()
    ret, frame2 = cap2.read()

    # Make detections
    pic = np.hstack((frame1, frame2))
    results = model(np.hstack((frame1, frame2)))
    results2 = model2(np.hstack((frame1, frame2)))

    a, b, c, d = 0, 0, 0, 0
    for detection in results.xyxy[0]:

        # Gauze detection
        if detection[5].item() in model.classes:
            if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                a += 1
            elif detection[2].item() > frame1.shape[1]:
                b += 1

    for detection in results2.xyxy[0]:
        # Human detection
        if detection[5].item() == 0:
            if detection[2].item() <= frame1.shape[1]:  ##xmax < middle line
                c += 1
            elif detection[2].item() > frame1.shape[1]:
                d += 1

    image = np.squeeze(results.render())

    # Adding black border at the bottom
    borderedFrame = cv2.copyMakeBorder(image, 0, 350, 640 - frame1.shape[1], 640 - frame2.shape[1], cv2.BORDER_CONSTANT,
                                       value=[0, 0, 0])

    if c == 0:
        tempcount1 = a
        if tempcount1 != onscreenIn:
            frameCountIn += 1
            if frameCountIn == condition:
                if tempcount1 > onscreenIn:
                    countIn += tempcount1 - onscreenIn
                onscreenIn = tempcount1
        else:
            frameCountIn = 0
        cv2.putText(borderedFrame, 'On Screen =' + str(onscreenIn), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + str(countIn), (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                    2, cv2.LINE_AA)
    else:
        cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + 'Paused', (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Human Detected', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

    if d == 0:
        tempcount2 = b
        if tempcount2 != onscreenOut:
            frameCountOut += 1
            if frameCountOut == condition:
                if tempcount2 > onscreenOut:
                    countOut += tempcount2 - onscreenOut
                onscreenOut = tempcount2
        else:
            frameCountOut = 0
        cv2.putText(borderedFrame, 'On Screen =' + str(onscreenOut), (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total Out =' + str(countOut), (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
    else:
        cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total Out =' + 'Paused', (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Human Detected', (840, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # FPS Counter
    frame_count += 1
    elapsed_time = time.time() - start_time
    if elapsed_time > 1: 
        avg_fps = frame_count / elapsed_time
        frame_count = 0
        start_time = time.time()
    cv2.putText(borderedFrame, "Avg FPS =" + str(round(avg_fps, 1)), (540, 520), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


    # Gauze In-Play Counter
    countPlay = countIn - onscreenIn - countOut
    cv2.putText(borderedFrame, 'In Play =' + str(countPlay), (360, 650), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4,
                cv2.LINE_AA)

    cv2.imshow('Gauze', borderedFrame)

    key = cv2.waitKey(10)
    if key & 0xFF == ord('0'):
        onscreenIn += 1
    elif key & 0xFF == ord('1'):
        onscreenIn -= 1
    elif key & 0xFF == ord('2'):
        countIn += 1 
    elif key & 0xFF == ord('3'):
        countIn -= 1
    elif key & 0xFF == ord('4'):
        onscreenOut += 1
    elif key & 0xFF == ord('5'):
        onscreenOut -= 1
    elif key & 0xFF == ord('6'):
        countOut += 1
    elif key & 0xFF == ord('7'):
        countOut -= 1
    elif key & 0xFF == ord('8'):
        countPlay += 1
    elif key & 0xFF == ord('9'):
        countPlay -= 1
    elif key & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
YOLOv5  2023-9-15 Python-3.7.16 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)



Exception: [Errno 2] No such file or directory: 'C:\\Users\\Tee Chang Zen\\Documents\\OneDrive\\UREKA Project\\IGuaze\\yolov5-master\\202318JUL.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.